In [1]:
from dash import Dash, html, dcc, Output, Input
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import pycountry
import pycountry_convert


In [2]:
trend = pd.read_csv('worldriskindex-trend.csv')
display(trend.head())

,WRI.Country,ISO3.Code,Year,W,E,V,S,C,A,S_01,...,AI_04a_Norm,AI_04a_Base,AI_04b_Norm,AI_04b_Base,AI_04c_Norm,AI_04c_Base,AI_05a_Norm,AI_05a_Base,AI_05b_Norm,AI_05b_Base
0,Afghanistan,AFG,2000,4.18,0.25,69.83,61.97,73.22,75.05,64.24,...,100.00,24,91.54,24,100.00,5,89.84,28.99,57.66,6.29
1,Albania,ALB,2000,7.23,2.38,21.96,22.58,44.37,10.57,52.20,...,19.81,97,19.81,97,27.40,91,48.84,1232.01,0.01,0.00
2,Algeria,DZA,2000,10.28,2.65,39.85,29.57,52.73,40.58,47.78,...,40.23,86,42.01,86,42.43,78,44.09,1750.31,16.64,0.30
3,Andorra,AND,2000,0.29,0.02,4.32,2.84,2.15,13.18,22.22,...,12.85,98,27.52,95,35.78,84,31.68,5783.29,55.91,5.75
4,Angola,AGO,2000,10.98,2.18,55.35,35.29,62.14,77.32,64.30,...,84.23,31,100.00,21,85.65,10,58.17,604.63,93.27,325.00


In [3]:
trend = trend.drop(list(trend.filter(regex='Norm')), axis=1)
trend.columns = trend.columns.str.replace("_Base", "")
display(trend.head())

,WRI.Country,ISO3.Code,Year,W,E,V,S,C,A,S_01,...,AI_02b,AI_02c,AI_03a,AI_03b,AI_03c,AI_04a,AI_04b,AI_04c,AI_05a,AI_05b
0,Afghanistan,AFG,2000,4.18,0.25,69.83,61.97,73.22,75.05,64.24,...,0.03,2.41,9323.50,13058.96,30884.23,24,24,5,28.99,6.29
1,Albania,ALB,2000,7.23,2.38,21.96,22.58,44.37,10.57,52.20,...,0.12,24.22,136.67,2802.11,2123.41,97,97,91,1232.01,0.00
2,Algeria,DZA,2000,10.28,2.65,39.85,29.57,52.73,40.58,47.78,...,0.02,17.72,161.04,1571.86,3186.61,86,86,78,1750.31,0.30
3,Andorra,AND,2000,0.29,0.02,4.32,2.84,2.15,13.18,22.22,...,26.46,9.29,3.79,680.31,87.55,98,95,84,5783.29,5.75
4,Angola,AGO,2000,10.98,2.18,55.35,35.29,62.14,77.32,64.30,...,0.01,2.09,14739.43,8834.26,38264.30,31,21,10,604.63,325.00


In [4]:
meta = pd.read_excel('worldriskindex-meta.xlsx')[['Code', 'Variable']]
rename = dict(zip(meta['Code'], meta['Variable']))
rename

{'Country': 'Country Name',
 'ISO3': 'Three Digit Country Code',
 'W': 'WorldRiskIndex',
 'E': 'Exposition',
 'EI_01': 'Earthquakes',
 'EI_01a': 'Annually Averaged Population Exposed To Strong Intensity (Peak Ground Acceleration 0.1 g Or Higher)',
 'EI_01b': 'Annually Averaged Population Exposed To Strong Intensity (Peak Ground Acceleration 0.1 g Or Higher)',
 'EI_01c': 'Annually Averaged Population Exposed To Severe Intensity (Peak Ground Acceleration 0.2 g Or Higher)',
 'EI_01d': 'Annually Averaged Population Exposed To Severe Intensity (Peak Ground Acceleration 0.2 g Or Higher)',
 'EI_01e': 'Annually Averaged Population Exposed To Extreme Intensity (Peak Ground Acceleration 0.4 g Or Higher)',
 'EI_01f': 'Annually Averaged Population Exposed To Extreme Intensity (Peak Ground Acceleration 0.4 g Or Higher)',
 'EI_02': 'Tsunamis',
 'EI_02a': 'Annually Averaged Population Exposed To Strong Intensity (Coastal Run-Up Height 1.0 m Or Higher)',
 'EI_02b': 'Annually Averaged Population Expose

In [5]:
fullAggregate = trend.groupby(trend['ISO3.Code']).agg(
    Risk = ('W', 'mean'),
    Exposure = ('E', 'mean'),
    Vulnerability = ('V', 'mean')
)

fullAggregate['Country'] = fullAggregate.index.map(lambda x: pycountry.countries.get(alpha_3=x).name)

def convert_ISO3_Continent(ISO3):
    try:
        ISO2 = pycountry.countries.get(alpha_3=ISO3).alpha_2
        code = pycountry_convert.country_alpha2_to_continent_code(ISO2)
        continent = pycountry_convert.convert_continent_code_to_continent_name(code)
        return continent
    except:
        return pd.NA

fullAggregate['Continent'] = fullAggregate.index.map(lambda x: convert_ISO3_Continent(x))

display(fullAggregate.head())

,Risk,Exposure,Vulnerability,Country,Continent
ISO3.Code,,,,,
AFG,4.027308,0.250000,64.964615,Afghanistan,Asia
AGO,10.652692,2.309615,49.459231,Angola,Africa
ALB,6.291154,2.329231,17.505385,Albania,Europe
AND,0.274231,0.020000,3.805000,Andorra,Europe
ARE,5.296923,3.715000,7.654231,United Arab Emirates,Asia


In [6]:
exposureSubCol = ["EI_0" + str(i+1) for i in range(7)]
disasterAggregate = trend.groupby(trend['ISO3.Code'])[exposureSubCol].mean()
aggregate = fullAggregate.join(disasterAggregate)

print(aggregate.head())

                Risk  Exposure  Vulnerability               Country Continent  \
ISO3.Code                                                                       
AFG         4.027308  0.250000      64.964615           Afghanistan      Asia   
AGO        10.652692  2.309615      49.459231                Angola    Africa   
ALB         6.291154  2.329231      17.505385               Albania    Europe   
AND         0.274231  0.020000       3.805000               Andorra    Europe   
ARE         5.296923  3.715000       7.654231  United Arab Emirates      Asia   

               EI_01     EI_02      EI_03      EI_04  EI_05      EI_06  \
ISO3.Code                                                                
AFG        56.568077  0.010000   0.010000  46.588846   0.01   2.460769   
AGO         0.120000  0.134231  25.241154  33.426154   0.01  56.613077   
ALB         2.674615  2.044231  60.068077  25.369615   0.01   0.174615   
AND         0.140000  0.010000   0.010000   0.010000   0.01   

In [7]:
stats = pd.read_csv('cleanedCountries.csv')
stats = stats.set_index('ISO3')
stats = stats.drop(['Country', 'Region'], axis=1)
display(stats.head())
len(stats)

,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
ISO3,,,,,,,,,,,,,,,,,,
AFG,31056997,647500,48.0,0.00,23.06,163.07,700.0,36.0,3.2,12.13,0.22,87.65,1.0,46.60,20.34,0.380000,0.240000,0.380000
ALB,3581655,28748,124.6,1.26,-4.93,21.52,4500.0,86.5,71.2,21.09,4.42,74.49,3.0,15.11,5.22,0.232000,0.188000,0.579000
DZA,32930091,2381740,13.8,0.04,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,96.53,1.0,17.14,4.61,0.101000,0.600000,0.298000
ASM,57794,199,290.4,58.29,-20.71,9.27,8000.0,97.0,259.5,10.00,15.00,75.00,2.0,22.46,3.27,0.175125,0.215250,0.608937
AND,71201,468,152.1,0.00,6.60,4.05,19000.0,100.0,497.2,2.22,0.00,97.78,3.0,8.71,6.25,0.044480,0.246083,0.714625


227

In [8]:
aggregate = aggregate.join(stats)

In [9]:
def formAggregate(lowerBound):
    exposureSubCol = ["EI_0" + str(i+1) for i in range(7)]
    
    # Filtering to be within bounds
    aggregate = trend[trend['Year'] >= lowerBound]
    
    # Years 
    aggregate = aggregate.groupby(aggregate['ISO3.Code']).agg(
        Risk = ('W', 'mean'),
        Exposure = ('E', 'mean'),
        Vulnerability = ('V', 'mean'))
    
    # Add country and continent
    aggregate['Country'] = aggregate.index.map(lambda x: pycountry.countries.get(alpha_3=x).name)
    aggregate['Continent'] = aggregate.index.map(lambda x: convert_ISO3_Continent(x))
    
    # Add disaster columns
    disasterAggregate = trend.groupby(trend['ISO3.Code'])[exposureSubCol].mean()
    aggregate = aggregate.join(disasterAggregate)

    # Add selected country stats
    aggregate = aggregate.join(stats)

    return aggregate


In [10]:
fullFig = px.choropleth(aggregate, locations=aggregate.index, color="Risk", hover_name="Country", hover_data={'Population': True, 'Area (sq. mi.)': True, 'GDP ($ per capita)': True, 'Literacy (%)': True, 'Phones (per 1000)':True, 'Birthrate': True, 'Deathrate': True}, color_continuous_scale="Viridis")
fullFig.update_layout(margin=dict(l=0,r=0,t=0,b=0))
fullFig.show()

In [11]:
def generateMap(dataFrame):
    fullFig = px.choropleth(dataFrame, 
                            locations=dataFrame.index, 
                            color="Risk", 
                            hover_name="Country", 
                            hover_data={'Population': True, 
                                        'Area (sq. mi.)': True, 
                                        'GDP ($ per capita)': True, 
                                        'Literacy (%)': True, 
                                        'Phones (per 1000)':True, 
                                        'Birthrate': True, 
                                        'Deathrate': True},
                            color_continuous_scale="Viridis")
    fullFig.update_layout(margin=dict(l=0,r=0,t=0,b=0))
    return fullFig

In [12]:
iqrExposure = aggregate['Exposure'].quantile([0.25, 0.5, 0.75])
iqrVulnerability = aggregate['Vulnerability'].quantile([0.25, 0.5, 0.75])
maxExposure = aggregate['Exposure'].max()
maxVulnerability = aggregate['Vulnerability'].max()

In [13]:
aggregate.describe()

,Risk,Exposure,Vulnerability,EI_01,EI_02,EI_03,EI_04,EI_05,EI_06,EI_07,...,Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
count,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,...,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000
mean,8.202599,5.326441,24.106495,18.856335,17.284757,20.795474,31.355452,4.197467,13.054432,28.618998,...,198.044359,14.653665,4.586492,80.759424,2.182251,23.028015,9.761699,0.161757,0.296958,0.540876
std,9.188130,10.073932,15.436195,24.519677,23.015150,24.088896,21.843101,15.665554,20.824798,21.045067,...,205.696725,13.587355,8.584580,16.594071,0.733955,11.437408,5.157042,0.149096,0.133619,0.149965
min,0.180000,0.020000,1.626923,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,...,0.200000,0.000000,0.000000,33.330000,1.000000,8.250000,2.410000,0.000000,0.040000,0.062000
25%,2.316154,0.242692,12.566154,0.120000,0.010000,0.010000,15.838462,0.010000,0.170000,0.010000,...,26.800000,3.675000,0.240000,70.395000,2.000000,12.805000,6.225000,0.043740,0.208500,0.424000
50%,4.209615,1.055000,19.347692,2.295385,0.149615,2.539615,31.286154,0.010000,2.390385,31.058462,...,127.100000,10.830000,1.250000,84.770000,2.000000,20.740000,8.280000,0.116000,0.276000,0.559000
75%,10.189231,5.215385,31.372308,44.782308,38.820385,37.609231,46.588846,0.170000,10.266538,43.923077,...,305.550000,21.440000,4.425000,94.820000,3.000000,31.480000,12.165000,0.233000,0.351000,0.658500
max,45.938077,64.423077,70.445385,82.658077,77.694615,92.590000,88.946154,89.595385,68.639231,77.093462,...,1035.600000,62.110000,50.680000,100.000000,4.000000,50.730000,29.740000,0.769000,0.906000,0.900000


In [14]:
fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = aggregate['Exposure']['CAN'],
    gauge = {'bar': {'color': 'black'},
             'axis': {'range': [0, maxExposure]},
             'steps': [
                 {'range': [0, iqrExposure[0.25]], 'color': "green"},
                 {'range': [iqrExposure[0.25], iqrExposure[0.5]], 'color': "yellow"},
                 {'range': [iqrExposure[0.5], iqrExposure[0.75]], 'color': "orange"},
                 {'range': [iqrExposure[0.75], maxExposure], 'color': "red"}
             ]
             }
    ),
    layout = {
            'title': 'Exposure Ind',
        }
    )
fig.show()

In [15]:
def generateExposureInd(ISO, dataFrame):
    continent = dataFrame['Continent'][ISO]
    continentDf = dataFrame.loc[dataFrame['Continent'] == continent,:]
    maxExposure = continentDf['Exposure'].max()
    iqrExposure = continentDf['Exposure'].quantile([0.25, 0.5, 0.75])

    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = dataFrame['Exposure'][ISO],
        gauge = {'bar': {'color': 'black'},
             'axis': {'range': [0, maxExposure]},
             'steps': [
                 {'range': [0, iqrExposure[0.25]], 'color': "green"},
                 {'range': [iqrExposure[0.25], iqrExposure[0.5]], 'color': "yellow"},
                 {'range': [iqrExposure[0.5], iqrExposure[0.75]], 'color': "orange"},
                 {'range': [iqrExposure[0.75], maxExposure], 'color': "red"}
             ]
             }
        ),
        layout = {
            'title': 'Exposure Indicator',
        }
    )
    return fig

def generateVulnerabilityInd(ISO, dataFrame):
    continent = dataFrame['Continent'][ISO]
    continentDf = dataFrame.loc[dataFrame['Continent'] == continent,:]
    maxVulnerability = continentDf['Vulnerability'].max()
    iqrVulnerability = continentDf['Vulnerability'].quantile([0.25, 0.5, 0.75])

    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = dataFrame['Vulnerability'][ISO],
        gauge = {'bar': {'color': 'black'},
             'axis': {'range': [0, maxVulnerability]},
             'steps': [
                 {'range': [0, iqrVulnerability[0.25]], 'color': "green"},
                 {'range': [iqrVulnerability[0.25], iqrVulnerability[0.5]], 'color': "yellow"},
                 {'range': [iqrVulnerability[0.5], iqrVulnerability[0.75]], 'color': "orange"},
                 {'range': [iqrVulnerability[0.75], maxVulnerability], 'color': "red"}
             ]
             }
        ),
        layout = {
            'title': 'Vulnerability Indicator',
        }
    )
    return fig

In [16]:
countryCode = 'CAN'

countryExp = aggregate.loc[countryCode, exposureSubCol].astype('float64')
continentExp = aggregate.loc[aggregate['Continent'] == aggregate.loc[countryCode, 'Continent'], exposureSubCol].mean()
exposure = pd.DataFrame({
    'country': countryExp,
    'continent': continentExp
})
exposure['disaster'] = exposure.index.map(lambda x: rename[x])

exposure

,country,continent,disaster
EI_01,24.541923,20.692458,Earthquakes
EI_02,44.020385,27.599247,Tsunamis
EI_03,40.849231,14.038545,Coastal Floodings
EI_04,53.362692,16.095184,Riverine Floodings
EI_05,2.452308,7.982040,Cyclones
EI_06,36.048462,11.982559,Droughts
EI_07,39.941154,36.081706,Sea Level Rise


In [17]:
exposureFig = px.bar(exposure, x='disaster', y=['country', 'continent'], barmode='overlay')
exposureFig.show()

In [18]:
def generateDisasterGraph(countryCode, dataFrame):
    countryExp = dataFrame.loc[countryCode, exposureSubCol].astype('float64')
    continentExp = dataFrame.loc[dataFrame['Continent'] == dataFrame.loc[countryCode, 'Continent'], exposureSubCol].mean()
    exposure = pd.DataFrame({
        'country': countryExp,
        'continent': continentExp
    })
    exposure['disaster'] = exposure.index.map(lambda x: rename[x])
    
    fig = px.bar(exposure, x='disaster', y=['country', 'continent'], barmode='overlay')
    return fig



In [19]:
continents = {
    "North America" : (55, -105),
    "South America" : (-9,-56),
    "Asia" : (34, 101),
    "Africa" : (9, 35),
    "Europe" : (55, 15),
    "Oceania" : (-23, 140)
}

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.Div(
        style={'display': 'flex'},
        children=[
            dcc.Dropdown(id='continents', 
                         options=[{'label': c, 'value': c} for c in continents], 
                         value='North America', 
                         style={'width': '25vw', 'height': '5vh'}),
            dcc.Dropdown(id='aggregate',
                         options=[{'label': 'Last 25 years', 'value':2000},
                                  {'label': 'Last 10 years', 'value':2015},
                                  {'label': 'Last 5 years', 'value':2020}],
                         value=2000,
                         style={'width': '25vw', 'height': '5vh'})
        ]
    ),
    
    dcc.Store(id='code'),
    dcc.Store(id='dataFrame', data=aggregate.to_dict()),
    dcc.Graph(id='map', style={'width': '50vw', 'height': '37.5vh'}, responsive=True),

    html.Div(
        style={'display': 'flex'},
        children=[
            dcc.Graph(id='exposure', style={'width': '25vw', 'height': '20vh'}, responsive=True),
            dcc.Graph(id='vulerability', style={'width': '25vw', 'height': '20vh'}, responsive=True)
        ]
    ),

    dcc.Graph(id='disaster', style={'width': '50vw', 'height': '37.5vh'}, responsive=True)
])

@app.callback(
    Output('dataFrame', 'data'),
    Input('aggregate', 'value')
)
def generateDataFrame(bound):
    return formAggregate(bound).to_dict()

@app.callback(
    Output('map', 'figure'),
    Input('continents', 'value'),
    Input('dataFrame', 'data')
)
def getMap(selected, data):
    df = pd.DataFrame.from_dict(data)
    values = continents[selected]
    fig = generateMap(df)

    fig.update_geos(
        center_lat=values[0],
        center_lon=values[1],
        projection_scale=3
    )

    return fig

@app.callback(
        Output('code', 'data'),
        Input('map', 'clickData')
)
def getISOCode(clickData):
    if clickData:
        ISO = clickData['points'][0]['location']
        return ISO
    return 'CAN'


@app.callback(
    Output('exposure', 'figure'),
    Input('code', 'data'),
    Input('dataFrame', 'data')
)
def getExposureInd(ISO, data):
    df = pd.DataFrame.from_dict(data)
    return generateExposureInd(ISO, df)
    
@app.callback(
    Output('vulerability', 'figure'),
    Input('code', 'data'),
    Input('dataFrame', 'data')
)
def getVulnerabilityInd(ISO, data):
    df = pd.DataFrame.from_dict(data)
    return generateVulnerabilityInd(ISO, df)

@app.callback(
    Output('disaster', 'figure'),
    Input('code', 'data'),
    Input('dataFrame', 'data')
)
def getDisasterGraph(ISO, data):
    df = pd.DataFrame.from_dict(data)
    return generateDisasterGraph(ISO, df)
        

if __name__ == '__main__':
    app.run(debug=True)


CAN
{'Risk': {'AFG': 4.0273076923076925, 'AGO': 10.652692307692309, 'ALB': 6.291153846153846, 'AND': 0.2742307692307692, 'ARE': 5.296923076923077, 'ARG': 15.535769230769231, 'ARM': 2.2434615384615384, 'ATG': 3.7223076923076923, 'AUS': 19.799230769230768, 'AUT': 1.2738461538461539, 'AZE': 2.566153846153846, 'BDI': 2.9707692307692306, 'BEL': 4.594615384615384, 'BEN': 1.6084615384615384, 'BFA': 1.6630769230769231, 'BGD': 26.139615384615386, 'BGR': 2.316153846153846, 'BHR': 0.928076923076923, 'BHS': 4.719230769230769, 'BIH': 2.5623076923076926, 'BLR': 0.768076923076923, 'BLZ': 7.5065384615384625, 'BOL': 3.1334615384615385, 'BRA': 11.807692307692308, 'BRB': 2.4669230769230768, 'BRN': 1.3723076923076922, 'BTN': 1.2757692307692308, 'BWA': 1.4026923076923077, 'CAF': 3.293846153846154, 'CAN': 19.181923076923077, 'CHE': 1.1400000000000001, 'CHL': 15.661923076923076, 'CHN': 34.42884615384615, 'CIV': 2.1092307692307695, 'CMR': 9.473846153846154, 'COD': 9.650384615384615, 'COG': 4.519230769230769, 